In [2]:
import numpy as np
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [3]:
xml_file = "./Dataset/VSoSLCSum.xml"  # file chứa dữ liệu XML

In [4]:
tree = ET.parse(xml_file)
root = tree.getroot()

docs, cmts, sums = [], [], []

for post in root.findall("post"):
    post_id = post.get("id")
    title = post.findtext("title", "").strip()

    # --- Summary ---
    for sent in post.findall(".//summary/sentences/sentence"):
        text = sent.find("content").text.strip()
        sums.append({"post_id": post_id, "text": text, "title": title})

    # --- Document ---
    for sent in post.findall(".//document/sentences/sentence"):
        text = sent.find("content").text.strip()
        label = sent.get("in_summary")
        docs.append(
            {"post_id": post_id, "text": text, "label": int(label), "type": "document"}
        )

    # --- Comments ---
    for sent in post.findall(".//comments//sentence"):
        text = sent.find("content").text.strip()
        label = sent.get("in_summary")
        cmts.append(
            {"post_id": post_id, "text": text, "label": int(label), "type": "comment"}
        )

df_docs = pd.DataFrame(docs)
df_cmts = pd.DataFrame(cmts)
df_sums = pd.DataFrame(sums)

In [5]:
print("Summaries:")
df_sums["sent_id"] = df_sums.groupby("post_id").cumcount() + 1
df_sums

Summaries:


,post_id,text,title,sent_id
0,yen-bai_vne_2,Bí_thư Tỉnh_uỷ Yên_Bái_Phạm_Duy_Cường được hàn...,Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái,1
1,yen-bai_vne_2,Ước_tính cả trăm ôtô chở người đến chia buồn...,Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái,2
2,yen-bai_vne_2,Căn nhà 3 tầng không đủ lớn để đón khách viếng...,Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái,3
3,yen-bai_vne_2,"Sáng nay , Bộ_trưởng Công_an Tô_Lâm , Bộ_trưởn...",Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái,4
4,yen-bai_vne_2,"Bà Phạm_Thị_Thanh_Trà , Chủ_tịch UBND tỉnh Yên...",Tình_cảm của hàng_xóm với Bí_thư Tỉnh_ủy Yên_Bái,5
...,...,...,...,...
2443,is_vne_30,Thật_là thất_vọng hết_chỗ_nói .,"IS bắn hạ trực_thăng quân_sự Syria , hai phi_c...",12
2444,is_vne_30,"Cảm_ơn những chiến sỹ anh_hùng , hi_sinh để ti...","IS bắn hạ trực_thăng quân_sự Syria , hai phi_c...",13
2445,is_vne_30,Mình chưa hiểu vai_trò của S300 mà Nga đặt ở S...,"IS bắn hạ trực_thăng quân_sự Syria , hai phi_c...",14
2446,is_vne_30,Mình không hiểu vậy hệ_thống S300 của Nga đặt ...,"IS bắn hạ trực_thăng quân_sự Syria , hai phi_c...",15


In [19]:
# print("Documents:")
# df_docs

In [20]:
# print("Comments:")
# df_cmts

In [6]:
df_contents = pd.concat([df_docs, df_cmts], ignore_index=True)
df_contents

,post_id,text,label,type
0,yen-bai_vne_2,"Sáng 19/8 , thành_phố Yên_Bái mưa không ngớt d...",2,document
1,yen-bai_vne_2,Ước_tính cả trăm ôtô chở người đến chia buồn...,3,document
2,yen-bai_vne_2,Căn nhà 3 tầng không đủ lớn để đón khách viếng...,3,document
3,yen-bai_vne_2,Nhiều cảnh_sát có_mặt tại đây để hướng_dẫn đỗ ...,0,document
4,yen-bai_vne_2,Người_dân đứng xem chật hai bên đường,1,document
...,...,...,...,...
10681,is_vne_30,FN - 6 quá mạnh khi vào tay IS,0,comment
10682,is_vne_30,Thật đáng tiếc,1,comment
10683,is_vne_30,Cần tiêu_diệt tận gốc is,1,comment
10684,is_vne_30,sự hy_sinh dũng_cảm,1,comment


In [7]:
sums_num = df_sums['post_id'].count()
docs_num = df_docs['post_id'].count()
cmts_num = df_cmts['post_id'].count()
print(f"Summaries: {sums_num:,}")
print(f"Documents: {docs_num:,}")
print(f"Comments: {cmts_num:,}")

Summaries: 2,448
Documents: 3,760
Comments: 6,926


In [8]:
rows = []

for pid, sum_grp in df_sums.groupby("post_id"):
    contents_grp = df_contents[df_contents["post_id"] == pid].reset_index(drop=True)
    
    for summ_row in sum_grp.itertuples(index=False):
        sent_id = summ_row.sent_id
        for cont_row in contents_grp.itertuples(index=False):
            rows.append({
                "post_id": pid,
                "sent_id": sent_id,
                "text": cont_row.text,
                "label": cont_row.label,
                "type": cont_row.type
            })

df_new = pd.DataFrame(rows)
df_new

,post_id,sent_id,text,label,type
0,casa_dantri_3,1,Lê_Kiêm_Toàn - Đại_tá Lữ_đoàn trưởng Lữ_đoàn 9...,5,document
1,casa_dantri_3,1,"Nguyễn_Đức_Hảo - Thượng_tá , Phi_đội trưởng Lữ...",5,document
2,casa_dantri_3,1,"Nguyễn_Văn_Chính - Thiếu_tá , Chính_trị_viên p...",5,document
3,casa_dantri_3,1,"Nguyễn_Ngọc_Chu - Thiếu_tá , phi_công kiêm dẫn...",4,document
4,casa_dantri_3,1,"Lê_Văn_Đình - Đại_úy , nhân_viên tuần_thám trê...",5,document
...,...,...,...,...,...
189275,yen-bai_zing_5,13,* 5 trường_hợp được nổ_súng không cần cảnh_báo...,2,document
189276,yen-bai_zing_5,13,"- Người đang sử_dụng vũ_khí , vật_liệu nổ tấn_...",2,document
189277,yen-bai_zing_5,13,"- Người đang sử_dụng vũ_lực , vũ_khí , vật_liệ...",2,document
189278,yen-bai_zing_5,13,- Người đang trực_tiếp thực_hiện hành_vi cướp ...,2,document


In [9]:
print("Content:", df_contents[df_contents['post_id'] == 'yen-bai_vne_2']['post_id'].count())
print("Summary:", df_sums[df_sums['post_id'] == 'yen-bai_vne_2']['post_id'].count())
print("Total:", df_new[df_new['post_id'] == 'yen-bai_vne_2']['post_id'].count())

Content: 55
Summary: 18
Total: 990


In [10]:
mkdir = "./Dataset/Done"
if not os.path.exists(mkdir):
    os.makedirs(mkdir)
df_sums.to_csv(os.path.join(mkdir, "summaries.csv"), index=False)
df_new.to_csv(os.path.join(mkdir, "contents.csv"), index=False) 
print("Files saved")

Files saved
